In [2]:
import shutil
import os

os.makedirs("/root/.config/kaggle", exist_ok=True)
shutil.copy("/kaggle/input/kaggle-api/kaggle.json", "/root/.config/kaggle/kaggle.json")

'/root/.config/kaggle/kaggle.json'

In [3]:
!chmod 600 /root/.config/kaggle/kaggle.json

In [ ]:
!kaggle kernels output "YOUR_MODEl_OUTPUT" -p /kaggle/working/dest1

In [5]:
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 82.3 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [6]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 58.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 11.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 98.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.3 MB/s eta 0:00:0

### Upload thư mục lora_model lên Hugging Face Hub rồi dùng tên repo

In [15]:
import os
print(os.path.isdir("/kaggle/working/dest1/lora_model"))

True


In [ ]:
from huggingface_hub import login
login(token="YOUR_TOKEN_HUGGINGFACE")

In [ ]:
from huggingface_hub import create_repo

# Thay đổi tên repo cho phù hợp
repo_id = "YOUR_PROJECT"  # <username>/<repo_name>
create_repo(repo_id, private=True)  # hoặc private=False nếu muốn công khai

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="/kaggle/working/dest1/lora_model",
    repo_id="YOUR_PROJECT",
    repo_type="model",
    token="YOUR_TOKEN_HUGGINGFACE",
)

In [ ]:
if True:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from unsloth import FastLanguageModel
    
    max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "YOUR_PROJECT", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

prompt = "<|im_start|>user\n{}\n<|im_end|>\n<|im_start|>assistant\n"

==((====))==  Unsloth 2025.4.7: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [31]:
origin_prompt = "Brand X soda advertises, ``We will give you 20$\\%$ more soda than Brand Y for a total price that is 10$\\%$ less than Brand Y's price!'' What is the ratio of the unit price of Brand X soda to the unit price of Brand Y soda? Express your answer as a common fraction."
inputs = tokenizer(
[
    prompt.format(
        origin_prompt,
    )
], return_tensors = "pt").to("cuda")

### Classification answer

Set up OpenAI API chatbot

In [ ]:
import openai
from openai import OpenAI
# Set your API key here

client = OpenAI(api_key = "YOUR OPENAI API KEY")

In [29]:
def evaluate_solution(problem, candidate_solution):
    prompt = f"""
You are a math expert helping evaluate the correctness of candidate solutions generated by another model.

First, analyze the following math problem and the provided solution carefully.

### Math Problem:
{problem}

### Candidate Solution:
{candidate_solution}

Now, step-by-step, reason about whether the solution is logically sound and reaches the correct final answer. Identify any mistakes or inconsistencies.

Finally, answer with one word only: "correct" if the reasoning is sound and the answer is accurate, or "incorrect" otherwise.

### Your reasoning:
"""

    messages = [
        {"role": "system", "content": "You are a helpful and accurate math evaluator."},
        {"role": "user", "content": prompt}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4"
        messages=messages,
        temperature=0.0,  # for deterministic judgment
    )

    result = response.choices[0].message.content.strip()

    if any(keyword in result for keyword in ["incorrect", "not correct"]):
        return 0
    else:
        return 1


In [ ]:
right_responses = []

while not right_responses:
    outputs = model.generate(**inputs, max_new_tokens = 2048, num_return_sequences=3, do_sample=True, use_cache = True)
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    for i, response in enumerate(responses, 1):
        response = response.split('<|im_start|>assistant', 1)[1]

        if evaluate_solution(origin_prompt, response):
            right_responses.append(response)

for idx, right_response in enumerate(right_responses):
    print("--------------------")
    print(f"[Response {idx}]:\n")
    print(response)

--------------------
[Response 0]:


Given that Brand X offers 20$\%$ more soda with a total cost that is 10$\%$ cheaper than Brand Y, we can analyze the information as follows:

Brand X price = 80$\%$ of Brand Y price
Brand Y price = 100$\%$ of Brand Y price

Brand X price = 0.8 * Brand Y price
Brand Y price = 100 * Brand Y price

We can find the ratio of the unit price of Brand X soda to the unit price of Brand Y soda by dividing the Brand X price by the Brand Y price:

Brand Y price = 100 * Brand Y price

To find the ratio, we divide the Brand X price by the Brand Y price:

Ratio = Brand X price / Brand Y price = (0.8 * Brand Y price) / (100 * Brand Y price)

Simplifying the fraction, we get:

Ratio = 0.8 / 100 = 0.8 \* 10^(-2) = 0.08 = \boxed{\frac{8}{100}}
The answer is: \frac{8}{100}
--------------------
[Response 1]:


Given that Brand X offers 20$\%$ more soda with a total cost that is 10$\%$ cheaper than Brand Y, we can analyze the information as follows:

Brand X price = 80$\